# [Advent of Code 2022 Day 12](https://adventofcode.com/2022/day/12)
dadada

## Initial setup

In [16]:
from __future__ import annotations
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Test Cases
dadada

### Part 1

In [17]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 31,
    },
    "input": {
        "input1": 361,
    },
}
PART_ONE_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

### Part 2

In [18]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 29,
    },
    "input": {
        "input1": 354,
    },
}
PART_TWO_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing
dadada

In [19]:
class Model(BaseModel):
    line: str

def parse_input_from_filename(filename: str) -> Context:
    lines = list(yield_line(filename))

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    for idx, line in enumerate(lines):
        input_lines.append(list(line))

    return ctx

### Test Parsing Examples

In [20]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
_________________________________ test_parsing_examples[example1] _________________________________
-------------------------------------- Captured stderr call ---------------------------------------
['S', 'a', 'b', 'q', 'p', 'o', 'n', 'm']
['a', 'b', 'c', 'r', 'y', 'x', 'x', 'l']
['a', 'c', 'c', 's', 'z', 'E', 'x', 'k']
['a', 'c', 'c', 't', 'u', 'v', 'w', 'j']
['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i']
1 passed in 0.02s


### Test Parsing Inputs

In [21]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
___________________________________ test_parsing_inputs[input1] ___________________________________
-------------------------------------- Captured stderr call ---------------------------------------
['a', 'b', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'c', 'c', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'a', 'a', 'a', 'a', 'a', 'a']
['a', 'b', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'

## Helper Functions

### Helper 1
dadada

### Get Height by Char

In [22]:
%%ipytest -xrPvvvvv

def get_height(char: str) -> int:
    assert len(char) == 1, f"Should only be a single char but gave {char}"
    if char == "S": return get_height("a")
    if char == "E": return get_height("z")
    if char not in "abcdefghijklmnopqrstuvwxyz":
        raise ValueError(f"Bad height char {char}")
    return ord(char) - ord("a") + 1

def test_get_height():
    assert get_height("a") == 1
    assert get_height("z") == 26
    assert get_height("S") == 1
    assert get_height("E") == 26

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


### Get Points with Filter

In [23]:
%%ipytest -xrPvvvvv

def get_points_with_filter(point: Point, kernel: list[tuple[int, int]], seen: set[Point]) -> list[Point]:
    result = []
    for disp in kernel:
        attempt = point + disp
        if attempt not in seen:
            result.append(attempt)
    return result

def test_get_points_with_filter() -> None:
    assert get_points_with_filter(Point(1, 1), DIR.VERT, {Point(0, 1)}) == [Point(2, 1)]

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


### Get Possible Points

In [24]:
%%ipytest -xrPvvvvv

def get_possible_points(grid: list[list[str]], curr_pos: Point) -> set[Point]:
    possible_moves = []
    rows = len(grid)
    cols = len(grid[0])
    x, y = curr_pos.to_tuple()
    curr_height = get_height(grid[x][y])
    for next_point in curr_pos.get_points(DIR.ADJA, 0, rows - 1, 0, cols - 1):
        nx, ny = next_point.to_tuple()
        next_height = get_height(grid[nx][ny])
        if (curr_height + 1) >= next_height:
            possible_moves.append(next_point)
    return set(possible_moves)

def test_get_possible_points() -> None:
    assert set(get_possible_points([["S", "a", "b"], ["a", "b", "c"]], Point(0, 0))) == {Point(0, 1), Point(1, 0)}
    assert set(get_possible_points([["S", "f", "b"], ["f", "b", "c"]], Point(0, 0))) == set()

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


### Solve Grid Brute Force
Same ill-hearted attempt as last year on Day 15 of AOC 2021... will I learn?

In [25]:
def solve_grid(grid: list[list[str]]) -> int:

    visited: set[Point] = {Point(0, 0)}
    record = INF

    def go(curr_pos: Point, curr_moves: int):
        nonlocal record
        x, y = curr_pos.to_tuple()
        if grid[x][y] == "E":
            record = min(record, curr_moves)
        for point in get_possible_points(grid, curr_pos):
            if point not in visited:
                visited.add(point)
                go(point, curr_moves + 1)
                visited.remove(point)

    go(Point(0, 0), 0)

    return record

### Dijkstra's
$$f(n)=g(n)$$ aka the backward cost

In [26]:
# Concretely implement the get_points method?
# Concretely implement the "get points not already visited" method?
# Fix hash, and other comparing things...
from abc import ABC, abstractmethod
class Problem(ABC):
    @abstractmethod
    def get_heuristic(self, node: Node) -> int:
        raise NotImplementedError(f"{self.__name__} does not implement get_heuristic()!")
    @abstractmethod
    def expand_node(self, node: Node) -> set[Node]:
        raise NotImplementedError(f"{self.__name__} does not implement expand_node()!")

class Node:
    def __init__(self, data: Any, depth: int = 0, parent: Node | None = None):
        self.data = data
        self.depth = depth
        self.parent = parent
    def __lt__(self, other):
        return self.data < other.data
    def __eq__(self, other):
        return self.data == other.data
    def __hash__(self):
        return hash(self.data)
    def __repr__(self):
        return str(self)
    def __str__(self):
        return f"Node(data={self.data}, depth={self.depth}, parent={self.parent})"

# Implement shortest-path tree
# Implement heap behaviour change
# Implement Node
#   Likewise, implement pathing in the node to parent
# Differentiate between returning path vs returning paths vs returning value
def best_first_search(grid: list[list[str]], start_node: Node) -> int:

    rows = len(grid)
    cols = len(grid[0])

    pushed: set[Node] = ImmutableSet({start_node})
    expanded: set[Node] = ImmutableSet()

    def is_feasible_jump(current: Point, desired: Point) -> bool:
        x, y = current
        curr_height = get_height(grid[x][y])
        x, y = desired
        next_height = get_height(grid[x][y])
        return (curr_height + 1) >= next_height

    def is_end_state(node: Node) -> bool:
        x, y = node.data
        return grid[x][y] == "E"

    def get_children(parent: Node) -> set[Node]:
        result = set()
        for child_point in parent.data.get_points(DIR.ADJA, 0, rows - 1, 0, cols - 1):
            child = Node(data=child_point, depth=parent.depth + 1, parent=parent)
            if is_feasible_jump(parent.data, child_point) and child not in pushed and child not in expanded:
                result.add(child)
        return result

    def get_heuristic_value(child: Node) -> int:
        return child.depth

    frontier = [(0, start_node)]

    while frontier:
        backward_cost, expanded_node = heappop(frontier)
        expanded.add(expanded_node)
        if is_end_state(expanded_node):
            return backward_cost
        for child_node in get_children(expanded_node):
            heuristic_value = get_heuristic_value(child_node)
            pushed.add(child_node)
            heappush(frontier, (heuristic_value, child_node))

    return INF

In [27]:
%%ipytest -xrPvvvvv
def return_sentinel_for_part(part: int) -> int:
    """
    Returns the proper sentinel for AOC.

    :param part: the round of AOC
    :return: the sentinel
    """
    if part == 1:
        return PART_ONE_SENTINEL
    if part == 2:
        return PART_TWO_SENTINEL
    raise NotImplementedError(f"Bad {part=}")

def test_return_sentinel_for_part() -> None:
    assert return_sentinel_for_part(1) == PART_ONE_SENTINEL
    assert return_sentinel_for_part(2) == PART_TWO_SENTINEL

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


## Main Function
dadada

In [28]:
def find_start(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == "S":
                return i, j

def solve(part: int, filename: str) -> int:
    stuff = parse_input_from_filename(filename).input
    if part == 1:
        disable_logging()
        return best_first_search(stuff, Node(Point(*find_start(stuff))))
    if part == 2:
        disable_logging()
        grid = stuff
        lmao = []
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == "a":
                    lmao.append((i, j))
        ans = INF
        for a, b in lmao:
            ans = min(ans, best_first_search(stuff, Node(Point(a, b))))
        return ans
    else:
        raise Exception(f"Invalid part: {part}")

## Execution

### Part 1

In [29]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 0.26s


### Part 2

In [30]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 10.34s


dadada